# Modeling Player Data

In [2]:
#Import Requried Libraires
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import seaborn as sns


import ipywidgets as ipw
import numpy as np
import pandas as pd
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players

import plotly as py
import plotly.graph_objects as go
import plotly.express as px
import plotly.tools as tls
from plotly.offline import iplot
import matplotlib.pyplot as plt
from bokeh.plotting import figure, output_notebook, show

pd.options.display.max_columns = None
py.offline.init_notebook_mode(connected=True)

## Player Career Stats

In [3]:
@interact(PlayerName='LeBron James',Stat=['PPG','APG','RPG', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PTS'])
def player_query (PlayerName,Stat): 
    player_dict = players.get_players()
   
    for player in player_dict:
        if player['full_name']==PlayerName:
            player_data=player
                        
    
    id = player_data['id']
    career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0]
    career['PPG']=(career['PTS']/career['GP']).round(1)
    career['APG']=(career['AST']/career['GP']).round(1)
    career['RPG']=(career['REB']/career['GP']).round(1)
    
    fig=px.scatter(career,x='SEASON_ID',y=Stat,color='TEAM_ABBREVIATION')
    fig.update_layout(xaxis_type='category')
     
    fig.update_layout(
        title=PlayerName+ "'s career " + Stat ,
        xaxis_title='Season',
        yaxis_title=Stat,
        legend_title='<b> Team </b>',
        template="plotly_dark",
        width = 975,
        
    )
    fig.show()
    
    for season in range(len(career)):
        print(f'Season {season+1}:{Stat} - {career[Stat][season]}')

   


interactive(children=(Text(value='LeBron James', description='PlayerName'), Dropdown(description='Stat', optio…

## Player All Time Stats

In [4]:
@interact(PlayerName='LeBron James')#Stat=['GP','GP','MIN','FGM','FGA','FG3M','FG3A','FTM', 'FTA','OREB','REB','AST','STL','BLK','TOV','PF','PTS','FGPCT','FG3PCT','FTPCT','RPG','ASG','PPG'])
def player_query (PlayerName): 

    player_dict = players.get_players()

    for player in player_dict:
         if player['full_name']==PlayerName:
            player_data=player

    id = player_data['id']
    career = playercareerstats.PlayerCareerStats(player_id=id).get_data_frames()[0]
    # Create a list of lists
    list_of_lists = [
        [PlayerName]
    ]
    # Create the DataFrame,'STL'
    df = pd.DataFrame(list_of_lists, columns = ['Player Name'])
    fields=['GP','GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','REB','AST','STL','BLK','TOV','PF','PTS']

    for i in range(len(fields)):
        df[fields[i]]=career[fields[i]].sum()
        df[fields[i]]=df[fields[i]].astype(int)

    df.insert(5, 'FGPCT', ((career['FGM'].sum()/career['FGA'].sum())*100).round(1))
    df.insert(8, 'FG3PCT', ((career['FG3M'].sum()/career['FG3A'].sum())*100).round(1))
    df.insert(11, 'FTPCT', ((career['FTM'].sum()/career['FTA'].sum())*100).round(1))
    df.insert(14, 'RPG', (career['REB'].sum()/career['GP'].sum()).round(1))
    df.insert(16,'ASG',(career['AST'].sum()/career['GP'].sum()).round(1))
    df.insert(21,'PPG',(career['PTS'].sum()/career['GP'].sum()).round(1))
    return df




interactive(children=(Text(value='LeBron James', description='PlayerName'), Output()), _dom_classes=('widget-i…

## Player Comparisons

In [6]:
@interact(PlayerName='LeBron James', PlayerName1='Michael Jordan')
def player_query (PlayerName,PlayerName1): 

    player_dict = players.get_players()
    player_data=[None]*2
#Search For player
    for player in player_dict:
        if player['full_name']==PlayerName:
            player_data[0]=player
        elif player['full_name']==PlayerName1:
            player_data[1]=player
            
    
#Query for player
    
    career=[None]*2
    for i in range(0,2):
        career[i] = playercareerstats.PlayerCareerStats(player_id=player_data[i]['id']).get_data_frames()[0]

    # Create a list of players
    list_of_lists = [
        [PlayerName],
        [PlayerName1]
    ]
    # Create the DataFrame
    data = pd.DataFrame(list_of_lists, columns = ['Player Name'])
    fields=['GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','REB','AST','STL','BLK','TOV','PF','PTS']
    for x in range(0,2):
        for i in range(len(fields)):
            data.at[x,fields[i]]=career[x][fields[i]].sum()
    #Convert to int
    for i in range(len(fields)):
        data[fields[i]]=data[fields[i]].astype(int)
    data.insert(5, 'FGPCT', None)
    data.insert(8, 'FG3PCT',None)
    data.insert(11, 'FTPCT',None)
    data.insert(14, 'RPG',None)
    data.insert(16,'APG',None)
    data.insert(21,'PPG',None)
    
   
    for x in range(0,2):
        data.at[x, 'FGPCT']=(((career[x]['FGM'].sum()/career[x]['FGA'].sum())*100).round(1))
        data.at[x,'FG3PCT']=(((career[x]['FG3M'].sum()/career[x]['FG3A'].sum())*100).round(1))
        data.at[x,'FTPCT']=(((career[x]['FTM'].sum()/career[x]['FTA'].sum())*100).round(1))
        data.at[x,'RPG']=((career[x]['REB'].sum()/career[x]['GP'].sum()).round(1))
        data.at[x,'APG']=((career[x]['AST'].sum()/career[x]['GP'].sum()).round(1))
        data.at[x,'PPG']=((career[x]['PTS'].sum()/career[x]['GP'].sum()).round(1))


    return data.style.apply(highlight_max)

def highlight_max(s): 
    is_max = s == s.max() 
    return ['background: lightgreen' if cell else '' for cell in is_max] 

#ADD COLOUR SLIDER FOR MORE PLAYERS

interactive(children=(Text(value='LeBron James', description='PlayerName'), Text(value='Michael Jordan', descr…

In [8]:
@interact(PlayerName='LeBron James', PlayerName1='Michael Jordan' ,Stat=['GP','MIN','FGM','FGA','FG3M','FG3A','FTM', 'FTA','OREB','REB','AST','APG','STL','BLK','TOV','PF','PTS','FGPCT','FG3PCT','FTPCT','RPG','ASG','PPG'])
def player_query (PlayerName,PlayerName1,Stat): 

    player_dict = players.get_players()
    player_data=[None]*2
#Search For player
    for player in player_dict:
        if player['full_name']==PlayerName:
            player_data[0]=player
        elif player['full_name']==PlayerName1:
            player_data[1]=player
            
    
#Query for player
    
    career=[None]*2
    for i in range(0,2):
        career[i] = playercareerstats.PlayerCareerStats(player_id=player_data[i]['id']).get_data_frames()[0]
    # Create a list of players
    list_of_lists = [
        [PlayerName],
        [PlayerName1]
    ]
    # Create the DataFrame,'STL'
    data = pd.DataFrame(list_of_lists, columns = ['Player Name'])
    fields=['GP','MIN','FGM','FGA','FG3M','FG3A','FTM','FTA','OREB','REB','AST','STL','BLK','TOV','PF','PTS']
    for x in range(0,2):
        for i in range(len(fields)):
            data.at[x,fields[i]]=career[x][fields[i]].sum()
    #Convert to int
    for i in range(len(fields)):
        data[fields[i]]=data[fields[i]].astype(int)
    data.insert(5, 'FGPCT', None)
    data.insert(8, 'FG3PCT',None)
    data.insert(11, 'FTPCT',None)
    data.insert(14, 'RPG',None)
    data.insert(16,'APG',None)
    data.insert(21,'PPG',None)
    
   
    for x in range(0,2):
        data.at[x, 'FGPCT']=(((career[x]['FGM'].sum()/career[x]['FGA'].sum())*100).round(1))
        data.at[x,'FG3PCT']=(((career[x]['FG3M'].sum()/career[x]['FG3A'].sum())*100).round(1))
        data.at[x,'FTPCT']=(((career[x]['FTM'].sum()/career[x]['FTA'].sum())*100).round(1))
        data.at[x,'RPG']=((career[x]['REB'].sum()/career[x]['GP'].sum()).round(1))
        data.at[x,'APG']=((career[x]['AST'].sum()/career[x]['GP'].sum()).round(1))
        data.at[x,'PPG']=((career[x]['PTS'].sum()/career[x]['GP'].sum()).round(1))
    #print(data)
    fig=px.bar(data,x='Player Name', y=Stat, color='Player Name', title=f'{Stat} Comparisons between {PlayerName} & {PlayerName1}')
    fig.show()
    

interactive(children=(Text(value='LeBron James', description='PlayerName'), Text(value='Michael Jordan', descr…